In [1]:
import os
import sys

import pandas as pd
import numpy as np
import yaml
import json

In [11]:
MayomicsVC_directory = '../../../../Mayomics-Research'
shell_script_dir = os.path.join(MayomicsVC_directory, 'MayomicsVC/src/shell')
os.listdir(shell_script_dir)

['mutect.sh',
 'vqsr.sh',
 'dedup.sh',
 'shared_functions.sh',
 'combine_variants.sh',
 'manta.sh',
 'trim_sequences.sh',
 'haplotyper.sh',
 'deliver_alignment.sh',
 'gvcftyper.sh',
 'phasing.sh',
 'merge_bams.sh',
 'deliver_haplotyperVC.sh',
 'alignment.sh',
 'GATK',
 'bqsr.sh',
 'realignment.sh',
 'shell_preamble.sh',
 'deliver_somaticVC.sh',
 'strelka.sh']

In [35]:
keys_to_ignore = ['h', 'd']
target_script = os.path.join(shell_script_dir, 'realignment.sh')
with open(target_script, 'r') as fh:
    lines = fh.readlines()
    
dosies = 0
caseies = 0
current_key = ''
casy_dictionary = {}
for line in lines:
    l = line.strip()
    
    if len(l) > 0 and not l[0] == '#':
        if 'case' in l: caseies += 1
        if 'esac' in l: caseies -= 1
        if caseies > 0:
            if ')' in l and not '(' in l and not '\?' in l and not ': )' in l:
                current_key = l.strip()
                if len(current_key) > 0 and not current_key[0] in keys_to_ignore:
                    current_key = current_key[0]
                    
            elif '=${' in l and len(current_key) > 0:
                v = l.strip().split('=')[0]
                if len(v) > 0:
                    casy_dictionary[current_key] = casy_dictionary
                    print(current_key, v)


s SAMPLE
b INPUTBAM
G REFGEN
k KNOWN
S SENTIEON
t THR
e ENV_PROFILE
F SHARED_FUNCTIONS


In [39]:
def get_bash_case_dict(fullfilename):
    
    keys_to_ignore = ['h', 'd']
    with open(fullfilename, 'r') as fh:
        lines = fh.readlines()
        
    caseies = 0
    current_key = ''
    casy_dictionary = {}
    for line in lines:
        l = line.strip()

        if len(l) > 0 and not l[0] == '#':
            if 'case' in l: caseies += 1
            if 'esac' in l: caseies -= 1
            if caseies > 0:
                if ')' in l and not '(' in l and not '\?' in l and not ': )' in l:
                    current_key = l.strip()
                    if len(current_key) > 0 and not current_key[0] in keys_to_ignore:
                        current_key = current_key[0]

                elif '=${' in l and len(current_key) > 0:
                    v = l.strip().split('=')[0]
                    if len(v) > 0:
                        casy_dictionary[current_key] = v

    return casy_dictionary

target_script = os.path.join(shell_script_dir, 'realignment.sh')
ca_dict = get_bash_case_dict(target_script)
for k, v in ca_dict.items():
    print('%20s:%s'%(k,v))

                   s:SAMPLE
                   b:INPUTBAM
                   G:REFGEN
                   k:KNOWN
                   S:SENTIEON
                   t:THR
                   e:ENV_PROFILE
                   F:SHARED_FUNCTIONS


In [19]:
for line in lines:
    l = line.strip()
    if len(l) > 0 and not l[0] == '#':
        if 'case' in l: caseies += 1
        if 'esac' in l: caseies -= 1
        if caseies > 0:
            print(l)


case ${OPT} in
h )  # Flag to display usage
echo -e "\n${DOCS}\n"
exit 0
;;
s )  # Sample name
SAMPLE=${OPTARG}
checkArg
;;
b )  # Full path to the input deduped BAM
INPUTBAM=${OPTARG}
checkArg
;;
G )  # Full path to reference genome fasta file
REFGEN=${OPTARG}
checkArg
;;
k )  # Full path to known sites files
KNOWN=${OPTARG}
checkArg
;;
S )  # Full path to sentieon directory
SENTIEON=${OPTARG}
checkArg
;;
t )  # Number of threads available
THR=${OPTARG}
checkArg
;;
e )  # Path to file with environmental profile variables
ENV_PROFILE=${OPTARG}
checkArg
;;
F )  # Path to shared_functions.sh
SHARED_FUNCTIONS=${OPTARG}
checkArg
;;
d )  # Turn on debug mode. Initiates 'set -x' to print all text. Invoked with -d
echo -e "\nDebug mode is ON.\n"
set -x
;;
\? )  # Check for unsupported flag, print usage and exit.
echo -e "\nInvalid option: -${OPTARG}\n\n${DOCS}\n"
exit 1
;;
: )  # Check for missing arguments, print usage and exit.
echo -e "\nOption -${OPTARG} requires an argument.\n\n${DOCS}\n